# Heart Disease Detection Model

## Introduction:

Heart disease is the second leading cause of death in Canada affecting over 1.2 million citizens (Canada 2022). Since heart disease affects more than 12% of the population and its obvious health consequences, a lot of research has been directed towards its treatment and prevention (Canada 2022). Although this has had great effects in decreasing the overall prevalence of heart disease in Canada since the early 2000s, there is still a major need to accurately detect the presence of heart disease in the population. 

The purpose of this project is to utilize features collected in cardiovascular health assessments to predict the presence of heart disease in a patient. To achieve this, we want to build a highly accurate classification model using historical data. Since this is a diagnosis classifier, one of our main objectives is to minimize the amount of false negative diagnoses. Our classifier's accuracy in identifying patients with heart disease is crucial due to the significant risks associated with the condition. 

In order to build this classifier, we will be using data from the heart disease dataset on UC Irvine’s Machine learning repository titled `cleveland.processed.data`. This dataset consists of 14 isolated features from 303 patients referred for coronary angiography at the Cleveland Clinic between May 1981 and September 1984 (Detrano et al., 1989). For this type of analysis, we will be building a classification model using the k-nearest neighbors’ classifier, which has been commonly used in the medical field as a simple but accurate classifier (Tayeb et al., 2017; Parry et al., 2010). In this approach, our new data point (patient) will be compared to all other data points we have in our dataset and to find the closest data points to our new one. The classification is determined by the average classification of the closest neighbors. As such, our classifiers development process will start by determining which attributes to use, then building a model using our training data. We will also utilize cross validation and determine which number of k neighbors’ outputs the highest accuracy.


## Method: 
Initially, according to our proposal, we built two different models, the first without having balanced data and the next having balanced data, based on 4 quantitative variables selected from the original 14 columns of our dataset. These 4 were chosen because of their quantitative nature and scientific collecting methods. The four predictor variables chosen were: 
1. `trestbps`: resting blood pressure (unit: mm Hg)
2. `chol`: serum cholesterol (unit: Milligrams per deciliter(mg/dl))
3. `thalach`: maximum heart rate achieved 
4. `oldpeak`: ST depression induced by exercise relative to rest (usually mm, measured on electrocardiogram (ECG or EKG)).

As for our response variable, we used the presence of heart disease(num). It takes 5 levels based on angiographic disease status where level 0 describes healthy patients and levels 1-4 describes patients diagnosed with increasing stages of heart disease.
However, these models are not suitable for clinical use as the recalls were too low, with 50.0% for the unbalanced model and 55.6% for the balanced model. The following method section will be based on the best model we trained with 13 predictor variables that were chosen by the original study among 76 collected variables. These included the following variables:
1. `age`: Age of the patient (years)
1. `sex`: Sex of the patient (Categorical with 2 levels- Male, Female)
1. `cp`: Chest pain type (Categorical with 4 levels-Type 1, Type 2, Type 3 and Type 4) Type 1: typical angina, Type 2: atypical angina, Type 3: non-anginal pain, Type 4: asymptomatic
1. `trestbps`: resting blood pressure (unit: mm Hg)
1. `chol`: serum cholesterol (unit: Milligrams per deciliter(mg/dl))
1. `fbs`: Fasting blood sugar > 120 mg/dl (Categorical with 2 levels-True,False)
1. `restecg`: Resting electrocardiographic results (Categorical with 3 levels-N(Normal), L1(Level 1), L2(Level 2))
1. `thalach`: Maximum heart rate achieved 
1. `exang`: Exercise induced angina (Categorical with 2 levels-Yes, No)
1. `oldpeak`: ST depression induced by exercise relative to rest (usually mm, measured on electrocardiogram (ECG or EKG)).
1. `slope`: The slope of the peak exercise ST segment (categorical with 3 levels-Up, Flat, Down)
1. `ca`: Number of major vessels (0-3) colored by flourosopy (Categorical with 4 levels-0, 1, 2, 3)
1. `thal`: The heart status as retrieved from Thallium test (Categorical with 3 levels-N(normal),FD(fixed defect), RD(reversible defect)

For our model training, we will use 75% of the entire dataset as a training set to train our classifier. We will ensure that our predictors are standardized, and the diagnosis presence (0-4) is balanced. Using a parameter grid with a range of k neighbor values, we will tune our model with $k$-fold cross validation. After fitting our predictors and target columns we will plot the $k$ value (x-axis) against the mean validation score (y-axis) to determine the $k$-value that gives us the highest accuracy. Finally, we will evaluate our model on the test set, by comparing the true diagnosis on the test set with our predictions to calculate accuracy. 

As a part of tuning the classifier and to visualize the result of whether or not our model has good accuracy, we will use a line plot to show the relationship between a range of K neighbours and accuracy estimates from the test datasets. 

### Expected Results:
We anticipate finding a relationship between our 4 predictor variables and the presence or absence of heart disease. This classifier could have major impacts on the early detection of heart disease, allowing for earlier treatment and better prevention, potentially saving lives. In addition, this project could provide a basis for further research in the development of more accurate classifying models of heart disease, e.g. by further investigation into relevant predictor variables.


### Dependencies
We first start off by loading dependencies below:

In [ ]:
!pip install pandas==1.5.3
!pip install scikit-learn==1.2.0
!pip install altair==4.2.2
!pip install matplotlib

In [2]:
import pandas as pd
import altair as alt
import numpy as np
import matplotlib.pyplot as plt 
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.metrics import confusion_matrix, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils import resample
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Simplify working with large datasets in Altair
alt.data_transformers.disable_max_rows()

# Output dataframes instead of arrays
set_config(transform_output="pandas")

set_config()

In [3]:
np.random.seed(seed=330)

### Loading dataset

Utilizing `pd.read_csv()` we can read in the dataset from  as detailed below. We will also isolate our predictors and target. We can also split it into a training and testing set.

In [4]:
col_names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'heart_disease_presence']
url = 'https://github.com/CCWebb14/DSCI100_Group_Project/blob/main/data/processed.cleveland.data?raw=true'

cleveland = pd.read_csv(url, names=col_names)
cleveland.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,heart_disease_presence
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


###  Dataset preprocessing

The data was already processed into 14 columns in total with 13 selected variables used for establishing a probability model in Detrano et al.’s study (1984). During the preprocessing of the dataset, rows with empty values were dropped, and levels of 1 to 4 indicated positive heart disease in the column heart_disease_presence were converted to 1 and the absence of heart disease remained as 0. The preprocessed dataset consisted of 297 observations with 14 columns.

In [5]:
def replace_values_greater_than_1(value):
    if value > 1:
        return 1
    else:
        return value

cleveland['heart_disease_presence'] = cleveland['heart_disease_presence'].apply(replace_values_greater_than_1)
cleveland.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,heart_disease_presence
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,1
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [6]:
cleveland = cleveland[~cleveland.isin(['?']).any(axis=1)]

Then, we randomly split the dataset into a 75% training set (222 observations) and a 25% testing set (75 observations). 

In [7]:
cleveland_train, cleveland_test = train_test_split(cleveland, test_size=0.25)

To build a classifier using K-nearest neighbours, it was essential to scale all the predictors, so standardizing the predictors is performed through preprocessing using the `StandardScaler()` function in the `scikit-learn` package.

In [8]:
cleveland_preprocessor = make_column_transformer(
    (StandardScaler(), ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']),
    remainder = 'passthrough',
    verbose_feature_names_out=False
)

cleveland_train_scaled = cleveland_preprocessor.fit_transform(cleveland_train)
cleveland_test_scaled = cleveland_preprocessor.fit_transform(cleveland_test)

#### Disease Presence Counts

The training set and testing set were transformed separately to prevent data leakage. The training set consisted of 119 observations of no presence of heart disease and 103 observations of positive presence of heart disease.

In [9]:
disease_presence_counts = pd.DataFrame(cleveland_train_scaled['heart_disease_presence'].value_counts()).reset_index().rename(
    columns={'index': 'heart_disease_presence', 'heart_disease_presence':'count'})

disease_presence_counts

,heart_disease_presence,count
0,0,119
1,1,103


This training dataset was balanced using the `resample()` method, so both the negative and positive presence of heart diseases have 119 observations.  

In [10]:
positive_heart_disease = cleveland_train_scaled[cleveland_train_scaled['heart_disease_presence'] == 1]
negative_heart_disease = cleveland_train_scaled[cleveland_train_scaled['heart_disease_presence'] == 0]

positive_heart_disease_upsample = resample(
    positive_heart_disease, n_samples=negative_heart_disease.shape[0]
)
upsampled_positive_heart_disease = pd.concat((positive_heart_disease_upsample, negative_heart_disease))
upsampled_disease_presence_counts = pd.DataFrame(upsampled_positive_heart_disease['heart_disease_presence'].value_counts()).reset_index().rename(
    columns={'index': 'heart_disease_presence', 'heart_disease_presence':'count'})
upsampled_disease_presence_counts

,heart_disease_presence,count
0,1,119
1,0,119


In [11]:
X_train = upsampled_positive_heart_disease.drop(columns = {'heart_disease_presence'})
y_train = upsampled_positive_heart_disease['heart_disease_presence']

Using a parameter grid with a range of 0 to 30 (exclusive) K-neighbours, we tuned our model with 10-fold cross-validation. `GridSearchCV()` method was used for tuning and scoring and was based on recall for clinical reasons to place focus on low false negatives. Then, a line graph (Figure 1) was generated after fitting the tuned grid, to show the accuracies across all numbers of neighbours in the range of 0 to 30. 


In [12]:
param_grid = {
    "kneighborsclassifier__n_neighbors": range(1, 40, 1),
}
cleveland_tune_pipe = make_pipeline(cleveland_preprocessor, KNeighborsClassifier())

In [13]:
knn_tune_grid = GridSearchCV(
    cleveland_tune_pipe, param_grid, cv=10
)

In [14]:
knn_model_grid = knn_tune_grid.fit(X_train, y_train)
accuracies_grid = pd.DataFrame(knn_model_grid.cv_results_) 

In [15]:
accuracy_versus_k_grid = (
    alt.Chart(accuracies_grid)
    .mark_line(point=True)
    .properties(title='Figure 1: Mean Test Score against K (Number of Neighbors)')
    .encode(
        x = alt.X('param_kneighborsclassifier__n_neighbors', title='Number of Neighbors', scale=alt.Scale(zero=False)),
        y = alt.Y('mean_test_score', title='Mean Test Score', scale=alt.Scale(zero=False))
    )
)
 
accuracy_versus_k_grid

alt.Chart(...)

#### Selecting $K$ and predicting heart disease presence in our test set
The `n_neighbours` value with the highest mean test score was chosen (n=19). We then evaluated our model's performance on the testing set, consisting of 75 observations, using the predict() function

In [16]:
knn_spec = KNeighborsClassifier(n_neighbors=19)
cleveland_fit = make_pipeline(cleveland_preprocessor, knn_spec).fit(X_train, y_train)

In [17]:
cleveland_test_predictions = cleveland_test_scaled.assign(
    predicted = cleveland_fit.predict(cleveland_test_scaled)
)
cleveland_test_predictions.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,heart_disease_presence,predicted
22,0.213487,0.771845,-0.949499,-0.824686,0.894102,-0.520756,0.980241,0.422524,-0.665062,0.631275,0.613559,-0.833663,-0.880854,1,0
53,-1.301117,0.771845,-0.949499,-0.242829,-0.495805,-0.520756,0.980241,1.618346,-0.665062,-0.877081,-0.973232,-0.833663,-0.880854,0,0
30,1.403533,-1.295597,-1.911828,0.339029,-0.068141,-0.520756,-1.033953,0.038152,-0.665062,0.631275,-0.973232,1.216328,-0.880854,0,0
274,0.321673,0.771845,-1.911828,-0.010086,-0.816553,-0.520756,-1.033953,0.507940,-0.665062,-0.206701,-0.973232,1.216328,-0.880854,1,0
1,1.187161,0.771845,0.975161,1.502745,0.936869,-0.520756,0.980241,-1.798289,1.503619,0.379882,0.613559,2.241324,-0.880854,1,1


#### Confusion Matrix

A confusion matrix (Figure 2) was generated to show four kinds of predictions generated with this model, as well as the precision and recall were calculated based on the prediction result from the testing set. 

In [18]:
y_true = cleveland_test_predictions['heart_disease_presence']
y_pred = cleveland_test_predictions['predicted']

<Figure size 800x600 with 0 Axes>

In [19]:
cm = confusion_matrix(y_true, y_pred)
cmd = ConfusionMatrixDisplay(cm, display_labels=['1','0'])
plt.figure(figsize=(8, 6)) 
cmd.plot()
plt.title('Figure 2: Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

#### Determining Model Performance

Finally, we were able to determine the accuracy, precision and recall of our classifier based on the number of correct predictions.

In [20]:
X_test = cleveland_test_scaled.drop(columns = {'heart_disease_presence'})
y_test = cleveland_test_scaled['heart_disease_presence']
cleveland_prediction_accuracy = cleveland_fit.score(X_test, y_test)

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

model_performance_df = pd.DataFrame({'Metric': ['Accuracy', 'Precision', 'Recall'],
        'Value': [cleveland_prediction_accuracy, precision, recall]}).round(3)

print('Table 1: Model Performance on Test Data')
model_performance_df

Table 1: Model Performance on Test Data


,Metric,Value
0,Accuracy,0.840
1,Precision,0.867
2,Recall,0.765


## Discussion
The goal of our project was to build an accurate classification model to predict the presence of heart disease based on available predictor features. We first decided to convert our response variable (heart disease presence level) from 0-4 to 0-1. As before, the range of 1-4 correlated to the severity of the disease but we decided to condense these all into the same category of positive diagnoses. Since the goal of our classifier is only detecting the presence of heart disease and not the severity of the disease, changing all the positive values to 1 and keeping all negatives as 0 was effective for the purpose of this classifier. 

Using the selected 4 predictor variables (`trestbps`, `chol`, `thalach`, `oldpeak`) from the proposal after changing the levels 1-4 to 1 in the heart disease presence variable, we obtained a high false negative value, recall of 50.0% and precision of 85.7%. To decrease the false negative value for clinical purposes, we balanced the samples from level 0 with 124 data counts and level 1 with 103 data counts to both levels having 124 data counts via upsampling. Through balancing the data, we increased the recall to 55.6%; however, precision was lowered to 69.0%. 
We then realized the difficulties in obtaining an ideal recall and lowering false positive cases may be due to the loss of information from dropping the categorical columns. Our original hypothesis that our classification model should only be trained on quantitative data resulted in an inaccurate classifier. By restoring all categorical columns (specified in the methods) and training the model on the balanced heart disease presence dataset, we obtained the final model with 84% accuracy, 86.7% precision and 76.5% recall (Table 1), which outcompeted the models trained with only 4 predictor variables. As such, we were able to build a highly accurate classifier. The 2x2 confusion matrix provides a clear representation of four kinds of prediction that this classifier can make as a disease classifier, and it's low false negative error rate (Figure 2).   

As mentioned previously, our goal is to have a classifier that can accurately predict whether or not a patient has heart disease. The current model is 84% accurate in predicting a randomly drawn testing set using K-nearest neighbours’ classifier with the number of neighbours equal to 19 (n = 19). The 84% prediction accuracy was obtained from a 10-fold cross-validation and n = 19 was determined by plotting the accuracies versus number of K-neighbours and obtaining the number of K that returned the highest prediction accuracy. This is an ideal accuracy that we are expected to find with current ability. Also, a current recall of 76.5% is higher than previous models trained with 4 predictor variables which indicates an increase in ratio between the number of correct positive predictions and total number of positive test set observations. Therefore, an increase in recall represents a decrease in false negatives, which benefits this model’s clinical utility.
	
The development of an accurate classification model for predicting the presence of heart disease based on various predictor variables could have impacts on clinical diagnosis improvement, early intervention and variable selection. The classifier with an 84% accuracy can potentially assist healthcare professionals in diagnosing the presence of heart disease more effectively. The model’s ability to balance precision (86.7%) and recall (76.5%) is crucial in a clinical setting. High precision ensures that positive predictions are reliable, while a high recall can indicate proportion of actual positive cases, reducing the chances of false negatives. A reliable heart disease prediction model can contribute to early intervention with early diagnosis. This could lead to a reduction in the severity of heart-related diseases. The 13 predictor variables used in this model were shown to have a relationship with heart disease presence, so they can be used as a guide to doctors when diagnosing a patient with heart disease.

Other impacts that our findings have on building clinical classifiers are consideration of categorial variables and trade-offs in precision and recall. The realization that including categorical variables in the model can improve its performance highlights the significance of considering various types of data in healthcare analytics. This insight could influence the design of future classification models for medical diagnoses. Our two abandoned models demonstrate the trade-offs between precision and recall and how balancing the dataset can impact model performance. By balancing the dataset, the precision dropped from 85% to 69% while recall increased from 50% to 55%. It is important for healthcare analysts to decide whether it’s more important to minimize false positives or false negatives based on the context of the medical application.
Since this classifier is 81% accurate, there is a great space for improvement, which could lead to further research, such as exploring additional predictor variables, investigating the generalizability of the model to different patient populations, and comparing with other existing classifiers.

## References:

1. Detrano, R., Janosi, A., Steinbrunn, W., Pfisterer, M., Schmid, J.-J., Sandhu, S., Guppy, K. H., Lee, S., & Froelicher, V. (1989). International application of a new probability algorithm for the diagnosis of coronary artery disease. The American Journal of Cardiology, 64(5), 304–310. https://doi.org/10.1016/0002-9149(89)90524-9
2. Canada, Public Health Agency of. “Government of Canada.” Canada.Ca, / Gouvernement du Canada, 28 July 2022, www.canada.ca/en/public-health/services/publications/diseases-conditions/heart-disease-canada.html.   
3. “Heart Disease.” UCI Machine Learning Repository, 1988, archive.ics.uci.edu/dataset/45/heart+disease.  
4. Parry, R M, et al. “K-nearest neighbor models for microarray gene expression analysis and clinical outcome prediction.” The Pharmacogenomics Journal, vol. 10, no. 4, 2010, pp. 292–309, https://doi.org/10.1038/tpj.2010.56. 
5. Tayeb, Shahab, et al. “Toward predicting medical conditions using K-nearest neighbors.” 2017 IEEE International Conference on Big Data (Big Data), 2017, https://doi.org/10.1109/bigdata.2017.8258395. 
